creating ml feature tables in postgres sql

In [19]:
import psycopg2

dbname = "ml_feature"
user = "postgres"
password = "12345"
host = "localhost"
port = 5432

# Connect to the existing database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Set isolation level to autocommit
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

with conn.cursor() as cursor:
    try:
        # Your table creation code
        cursor.execute("""
            CREATE TABLE numerical_features (
                id SERIAL PRIMARY KEY,
                reply_count INT,
                reply_users_count INT
            );
        """)

        cursor.execute("""
            CREATE TABLE categorical_features (
                id SERIAL PRIMARY KEY,
                msg_type VARCHAR(255),
                sender_name VARCHAR(255),
                msg_dist_type VARCHAR(255),
                channel VARCHAR(255),
                topic VARCHAR(255)
            );
        """)

        cursor.execute("""
            CREATE TABLE text_features (
                id SERIAL PRIMARY KEY,
                msg_content TEXT
            );
        """)

        cursor.execute("""
            CREATE TABLE sentimental_features (
                id SERIAL PRIMARY KEY,
                sentiment_score FLOAT
            );
        """)

        cursor.execute("""
            CREATE TABLE main_data (
                id SERIAL PRIMARY KEY,
                numerical_id INT REFERENCES numerical_features(id),
                categorical_id INT REFERENCES categorical_features(id),
                text_id INT REFERENCES text_features(id),
                sentimental_id INT REFERENCES sentimental_features(id),
                msg_sent_time TIMESTAMP,
                time_thread_start TIMESTAMP,
                tm_thread_end TIMESTAMP
            );
        """)

        print("Tables created successfully.")
    except psycopg2.Error as e:
        print(f"Error creating tables: {e}")

# Commit changes and close the connection
conn.commit()
conn.close()


Tables created successfully.


Load all the relevant features into the database

In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine


numerical_features_df = pd.read_csv("numerical_features.csv")
categorical_features_df = pd.read_csv("categorical_features.csv")
text_features_df = pd.read_csv("text_features.csv")
sentimental_features_df = pd.read_csv("sentimental_features.csv")


ml_feature_df = pd.concat([
    numerical_features_df,
    categorical_features_df,
    text_features_df,
    sentimental_features_df
], axis=1)

dbname = "ml_feature"
user = "postgres"
password = "12345"
host = "localhost"
port = 5432


engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")


ml_feature_table_name = "ml_feature_data"


ml_feature_df.to_sql(ml_feature_table_name, engine, if_exists='replace', index=False)

print("Combined features inserted into the PostgreSQL table successfully.")
